<a href="https://colab.research.google.com/github/cm-int/machine-learning-fundamentals/blob/main/module_2/Labs/Lab2_2_MobilePhonePrices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2.2: Creating a Multi-class and Multi-Label Machine Learning Model

In the first part of this lab, you'll create multi-class machine learning models to classify mobile phones into one of four different price brackets according to the features that the phones provide. You'll import, prepare, and visualize the data, and then you'll create machine learning models using the Decision Tree, Logistic Regression, and K-Nearest Neighbors algorithms. You'll compare the results of each of these models.

In the second part of this lab, you'll turn the problem around 180 degrees. You'll build a multi-label model that predicts whether a phone has 4G, WiFi, and Bluetooth capabilities based on its other characteristics, including the price bracket.

The solution code for this lab is available <a href="https://colab.research.google.com/github/cm-int/machine-learning-fundamentals/blob/main/module_2/Labs/Lab2_2_MobilePhonePrices_solution.ipynb" target="_parent">here</a>

#Read the Data

In [ ]:
# Upload the mobilephonedata.csv file from Github

!wget 'https://raw.githubusercontent.com/cm-int/machine-learning-fundamentals/main/module_2/Labs/mobilephonedata.csv'

In [ ]:
# Read the data into a Pandas DataFrame named mobilephonedata



#Split the Data

In [ ]:
# Create the phone_features DataFrame with every column apart from price_range. Display the results.



In [ ]:
# Create the phone_price DataFrame containing only the price_range column. Display the results.



In [ ]:
# Visualize the data to establish whether classification looks feasible
# NOTE: There are four price categories (0 to 3) in the price_range feature. The graph should create scatter plots for each category



#Use a Decision Tree to classify the data

In [ ]:
# Split the data into training and test datasets



In [ ]:
# Create and fit a Decision Tree classifier to the training data



In [ ]:
# Examine the decision tree generated by the classifier



In [ ]:
# Make predictions using the test data



In [ ]:
# Compare the test results from the previous step to the predicted results


In [ ]:
# Generate the confusion matrix
# Set the display labels to 'Price 0', 'Price 1', 'Price 2', and 'Price 3'


In [ ]:
# Calculate and display the accuracy, precision, recall, and error rate for the model



In [ ]:
# Calculate and display the probabilities for each prediction



In [ ]:
# Binarize the predictions test data (convert from a single column containing 0-3, to 4 columns containing 0 or 1)



In [ ]:
# Plot the ROC curve for each class
# NOTE: Don't worry about calculating the AUC

# Use Logistic Regression to classify the data

In [ ]:
# Repeat the steps as before and compare the results
# Use the same training and test DataFrames

# Create and fit a Logistic Regression classifier to the training data


In [ ]:
# Make predictions using the test data

In [ ]:
# Display the confusion matrix and compare it to the matrix for the Decision Tree

In [ ]:
# Calculate and display the accuracy, precision, score, and error rate for the model

In [ ]:
# Calculate and display the probabilities for each prediction

In [ ]:
# Display the ROC curve for each class

#Use the K-Nearest Neighbours algorithm to classify the data

In [ ]:
# Repeat the steps as before and compare the results
# Use the same training and test DataFrames

# Create and fit a K-Nearest Neighbors classifier to the training data

In [ ]:
# Make predictions using the test data

In [ ]:
# Display the confusion matrix and compare it to the matrices for the previous two models

In [ ]:
# Calculate and display the accuracy, precision, score, and error rate for the model

In [ ]:
# Calculate and display the probabilities for each prediction

In [ ]:
# Display the ROC curve for each class

# Perform Multi-label Classification
Predict whether a mobile phone has 4G, WiFi, and Bluetooth capabilities based on its other features and price category.

In [ ]:
# Retrieve the original data again

import numpy as np
import pandas as pd

mobilephonedata = pd.read_csv("mobilephonedata.csv")
mobilephonedata

In [ ]:
# Create the phone_features DataFrame with every column apart from blue, four_g, and wifi



In [ ]:
# Create the phone_labels DataFrame containing the blue, four_g, and wifi columns



In [ ]:
# Split the data into training and test datasets



In [ ]:
# Create a Multi Output Classifier wrapped around a Gradient Boosted Tree to classify the data

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier


# Show the number of estimators created for the model. There should be one classifier per label


In [ ]:
# Make test predictions



In [ ]:
# Generate the confusion matrices from the predictions
# Note: One confusion matrix per label


In [ ]:
# Calculate the accuracy, precision, and recall for the model



In [ ]:
# Calculate the probabilities for each prediction
 


In [ ]:
# Generate the ROC curves for each label
# How good is this model?
